**Caso de Viga sometida a carga:**

condiciones de frontera

y(0)=0

y(L)=0

Dominio [0, L]


In [29]:
#Librerias
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp

In [30]:
# Parámetros del problema
E = 30 * 1000  # Pasar de Ksi a psi (módulo de elasticidad)
I = 800  # en in^4 (momento de inercia de la viga)
w = 1 * 1000 / 12  # pasar de kip/ft a lb/in (carga distribuida)
L = 10 * 12  # pasar de pies a pulgadas (longitud de la viga)
n=100 #Número de nodos en la malla

In [31]:
#Resolver por Método de disparo lineal
# 1 Definir la ecuación diferencial
def ecuacion(x, y):
  return [y[1], (w * L * x - w * x**2) / (2 * E * I)] # y[1] es la derivada de la deflexión


In [32]:
# Condiciones de frontera
def c_frontera(y1, y2):
  return [y1[0], y2[0]]  # y(0) = 0 y y(L) = 0

#Arreglo de puntos
t_puntos = np.linspace(0, L, 100)


In [33]:
# Inicializar las condiciones de frontera
y0 = np.zeros((2, len(t_puntos)))
y0[0, 0] = 0  # Condición inicial y(0) = 0
y0[1, 0] = 1  # Condición inicial arbitraria para y'(0)

In [34]:
def disparo_lineal(): # Resolver utilizando solve_bvp
  sln = solve_bvp(ecuacion, c_frontera, t_puntos, y0)
  return sln.y[0]

resultados_disparo = disparo_lineal()

In [35]:
def dif_finitas(n):
  dx = L / (n + 1)  # Paso de la malla
  x_finitas = np.linspace(0, L, n + 2)

  sln_finitas = np.zeros(n + 2)  # Vector de resultados
  A = np.zeros((n, n))  # Matriz del sistema
  b = np.zeros(n)  # Vector de términos independientes

  # Llenar la matriz A y el vector b
  for i in range(n):
    A[i, i+1] = 2 * E * I / dx**2  # Diagonal principal
    if i > 0:
      A[i,i-1] = -E * I / dx**2  # Diagonal inferior
    if i < n - 1:
      A[i, i ] = -E * I / dx**2  # Diagonal superior
      b[i] = w * dx**2 / (E*I)


  # Resolver el sistema de ecuaciones
  sln_finitas[1:-1] = np.linalg.solve(A, b)

  # Deflexión 0 en los extremos
  sln_finitas[0] = 0
  sln_finitas[-1] = 0

  return sln_finitas

In [36]:
# Solución exacta dada en el problema para la deflexión de la viga
def sln_exacta(x):
  return (w * L * x**3) / (12 * E * I) - (w * x**4) / (24 * E * I) - (w * L**3 * x) / (24 * E * I)

In [37]:
# Gráfica
x_disparo = t_puntos
y_disparo = resultados_disparo

x_finitas, y_finitas = dif_finitas(n)

x_exacta = np.linspace(0, L, 100)
y_exacta = sln_exacta(x_exacta)

plt.plot(x_exacta, y_exacta, label="Solución Exacta", color="r")
plt.plot(x_finitas, y_finitas, label="sln Diferencias Finitas", color="green")  # Cambiado a "red"
plt.plot(x_disparo, y_disparo, label="sln Disparo Lineal", linestyle="--", color="b")
plt.title("Deflexión de la Viga")
plt.legend()
plt.grid(True)
plt.show()


IndexError: index 100 is out of bounds for axis 1 with size 100